In [2]:
import re

# 3rd part imports
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from dashboard.logic.io import read_gsheet, comment_button, total_assets, \
    total_value_to_num
from dashboard.logic.constants import GSHEETS_URL

# Forecasts

In [133]:
def set_bg_color(val, cmap: dict) -> str:
    """Map colors to DF values based on mapping dictionary.

    Args:
        val ( any type): Any value type.
        cmap (dict): Dict of the form {val:'color'}
    """
    return f'background-color: {cmap[val]}'

cmap = {i:c for i,c in zip(df_risks['Risk Level in 2023'], df_risks['color'])}



# print((df_risks.style
#     .applymap(set_bg_color, cmap=cmap, subset=['Risk Level in 2023'])
#     .set_properties(subset=pd.IndexSlice[4,'Risk Level in 2023'], **{"border-top-right-radius": "0.6em"})
#     .hide(axis='index')
# ).to_html())

In [122]:
from dashboard.logic.constants import nav_names
from dashboard.logic.io import read_gsheet, getDFs
from bokeh.palettes import inferno

def riskPallette(series: pd.Series, scale: dict) -> pd.Series:
    """Apply color based on risk level (# between 0-100).

    Args:
        scale (dict): Dictionary of the form {0:color, ... , 9:color}

    Returns:
        pd.Series: color
    """
    if series >= 90: 
        return scale["9"]
    if series < 10: 
        return scale["0"]
    else: 
        return scale[str(series)[0]]

def get_risk_pallete(pallette: dict) -> dict['int':'color']:
    """Generate risk pallete in scale 0 to 100 in steps of 10.
    Args:
        pallette (dict): Dictionary {n: ['colors'....]}
    Returns:
        Dict: Dictionary {"0": 'color'}
    """
    return {str(i):color for i,color in enumerate(pallette(10)[::-1])}

FORECASTS_URL = nav_names['2023 Forecasts & Risks']['url']

# read spreadsheet in as DF
df_fore_raw = read_gsheet(url=FORECASTS_URL, header=None)

# extract data into sub DF-s
references_dict = {
    'forecasts' : ("contains", "Forecasts", "down"),
    'risks' : ("equals", "Risks", "down")
}
df_dict_fore = getDFs(df_fore_raw, references_dict)

# FORECASTS
df_fore = df_dict_fore['forecasts'].copy()
df_fore.columns = df_fore.iloc[0]
df_fore = df_fore.iloc[1:].reset_index(drop=True)

# RISKS
risks_scale = {str(i):color for i,color in enumerate(inferno(10)[::-1])}

df_risks = df_dict_fore['risks'].copy()
df_risks.columns = df_risks.iloc[0]
df_risks = df_risks.iloc[1:].reset_index(drop=True)
df_risks['numeric_risk'] = (df_risks
    .filter(regex=(r"[Rr]isk\s[Ll]evel.*"))
    .squeeze()
    .str.extract(r"(\d+)")
    .astype('float64')
)
df_risks = df_risks.sort_values('numeric_risk', ascending=False, ignore_index=True)
df_risks['color'] = df_risks.numeric_risk.apply(riskPallette, scale=get_risk_pallete(inferno))


df_risks


16,Risks,Risk Level in 2023,numeric_risk,color
0,Recession,90%,90.0,#000003
1,Stock Market Crash 40%+,65%,65.0,#781C6D
2,Cyber Pandemic,50%,50.0,#A42C60
3,Housing Crash 20%+,35%,35.0,#ED6825
4,Bank Bail Ins,30%,30.0,#ED6825


# STOCKS

In [186]:
def findRefRowCol(df: pd.DataFrame, pattern: str, method: str='contains') -> tuple[int, int]:
    """Find reference row and column numeric index values.

    Args:
        pattern (str): Character sequence or regular expression.
        method (str, optional): Finding reference via pattern within the text ('contains')
            or equalling the value exactly ('equals'). Defaults to 'contains'.

    Raises:
        ValueError: Raises error if other than ['contains', 'equals'] is specified for the method.

    Returns:
        tuple[int, int]: Tuple of ['row_i', 'col_i']
    """
    
    
    # validate that method is correctly entered
    if method not in ['contains', 'equals']:
        raise ValueError(f"{method} can take only values: 'contains' or 'equals'!")
    
    idx_series = None
    if method == 'contains':
        idx_series = df.apply(lambda x: x.str.contains(pattern)).idxmax()
    if method == 'equals':
        idx_series = (df == pattern).idxmax()
    
    return idx_series.max(), idx_series.idxmax() 
    
def sliceDF(df: pd.DataFrame, 
            row_idx1: int=None, 
            row_idx2:int=None, 
            col_idx1: int=None, 
            col_idx2: int=None,
            col_0: bool=True, 
            direction='down') -> pd.DataFrame:
    """Slice DF till the first occuring empty row in given direction.

    Args:
        col_0 (bool, optional): If col_idx1 is actually first column. Defaults to True.
        direction (str, optional): Slice upwards or downwards from given row index. Defaults to 'down'.

    Raises:
        ValueError: If no row indices are specified.

    Returns:
        pd.DataFrame
    """
    
    # assert that at least one of the row indices is specified
    if row_idx1 is None and row_idx2 is None:
        raise ValueError(f"Both row indices can't equal {None}!") 
    
    # if column index is not the first column then None
    col_idx1 = col_idx1 if col_0 is True else None
    
    # find missing row index
    if direction == 'down':
        nan_mask = (df.iloc[row_idx1:,col_idx1:col_idx2] == 'nan').all(axis='columns')
        row_idx2 = None if nan_mask.sum() == 0 else nan_mask.idxmax()
        
    
    if direction == 'up':
        nan_mask = (df.iloc[:row_idx2,col_idx1:col_idx2] == 'nan').all(axis='columns')
        row_idx1 = None if nan_mask.sum() == 0 else nan_mask[::-1].idxmax() + 1
        row_idx2 += 1
         
    return df.iloc[row_idx1:row_idx2,col_idx1:col_idx2]

def getDFs(df: pd.DataFrame, references_dict: dict[str, tuple[str,str,str]]) -> dict[str, pd.DataFrame]:
    """Get subset DFs from a bigger DF based on reference strings.

    Args:
        df (pd.DataFrame): Raw initial DF.
        references_dict (dict): Dictionary of df_name: tuple('method', 'string' , 'direction', int('col_idx1)).
            method has 2 options ['contains' and 'equals'], 'direction' has 3 options 
            ['one', 'down', 'up'] where one means only one line needs to be parsed, 'up' and 'down'
            respectively correspond to the parsing direction from the reference point.
            
            Returns:
        dict: Dictionary of {'df_name': df}
    """
    
    
    # set all cols str type, NaN -> 'nan'
    df = df.reset_index(drop=True).astype(str)
    
    # find reference indices
    dfs = {}
    for k,v in references_dict.items():
        
        # find ref position
        row_i, col_i = findRefRowCol(df, pattern=v[1], method=v[0])
        
        # shift ref position if specified
        col_i = v[3] if len(v) == 4 else col_i
        
        if v[2] == 'one':
            dfs[k] = df.iloc[row_i,col_i:]
        if v[2] == 'down':
            dfs[k] = sliceDF(df, row_idx1=row_i, col_idx1=col_i, direction=v[2])
        if v[2] == 'up':
            dfs[k] = sliceDF(df, row_idx2=row_i, col_idx1=col_i, direction=v[2])

    # strip all NaN cols and rows
    dfs_clean = {}
    for k,df_ in dfs.items():
        
        if isinstance(df_, pd.Series):
            dfs_clean[k] = (df_
            .replace({'nan': np.nan})
            .dropna(how='any', axis='rows')
            )
        else:
            dfs_clean[k] = (df_
                .replace({'nan': np.nan})
                .dropna(how='all', axis='columns')
                .dropna(how='all', axis='rows')
            )
    
    return dfs_clean

In [192]:
STOCKS_URL = nav_names['Stocks']['url']

# read spreadsheet in as DF
df_stocks_raw = read_gsheet(url=STOCKS_URL, header=None)

# extract data into sub DF-s
references_dict = {
    'stocks' : ("equals", "Company", "down"),
    'analysis' : ("contains", "Analysis ratio", "down"),
    'sectors1' : ("contains", "S&P500 Index", "down"),
    'sectors2' : ("contains", "Don't just buy crap! ", "up", 0)
}
df_dict_stocks = getDFs(df_stocks_raw, references_dict)

df_dict_stocks.keys()

dict_keys(['stocks', 'analysis', 'sectors1', 'sectors2'])

In [191]:
def getNonSelectedDF(
    df_raw: pd.DataFrame, 
    df_dict: dict['df_name': pd.DataFrame], 
    start_idx: int=0, 
    stop_idx: int=None) -> pd.DataFrame:
    """Get data into DF that wasn't captured based on negative df_dict. 

    Args:
        df_raw (pd.DataFrame): Original DF where data wasn't cpatured.
        df_dict (_type_): Dict of the form {subdf_name: 'method', 'string' , 'direction', int('col_idx1))}.
        start_idx (int, optional): Starting row index (not positional).
        stop_idx (int, optional): Ending row index (not positional).

    Returns:
        pd.DataFrame: DF with rest of the info captured and NaN rows/cols stripped.
    """
    
    
    # find indicies already capture by "getDFs" function
    indices = np.array([])
    for df_ in df_dict.values():
        indices = np.concatenate((indices, df_.index.values))
    indices = [int(i) for i in indices]
    
    # subset DF
    df_sub = (df_raw
        .loc[~df_stocks_raw.index.isin(indices),:]
        .loc[start_idx:stop_idx,]
        .dropna(axis='columns', how='all')
        .dropna(axis='rows', how='all')
    )
    
    return df_sub
    
    
getNonSelectedDF(df_stocks_raw, df_dict_stocks, stop_idx=18)

,0
0,The absolute easiest strategy for stock market...
2,Start looking to move away from (Growth & Tech...
15,"Usually for a super safe play, I would suggest..."
16,I would suggest that since market growth is no...
17,"...reverse course, they will go back to QE & l..."


In [254]:
### STOCKS TABLE ###
df_stocks = df_dict_stocks['stocks'].copy().reset_index(drop=True)

# set column headers
df_stocks.columns = df_stocks.iloc[0]
df_stocks = df_stocks.iloc[1:].reset_index(drop=True)

# check if df is empty and if so is there extra info
all_nan = df_stocks.iloc[:,1:].isna().all(axis=1)
all_nan_idx = all_nan.loc[all_nan].index


df_stocks_info = df_stocks.loc[all_nan].dropna(axis='columns') # info
df_stocks = df_stocks.loc[~df_stocks.index.isin(all_nan_idx),] # df stocks

In [336]:
from dashboard.logic.io import comment_button

df_ana = df_dict_stocks['analysis'].copy()

# title of the analysis section
ana_title = df_ana.iloc[0,0]

# set header
header_idx = findRefRowCol(df_ana.astype('str'), r"Ratio:", 'contains')[0]
df_ana.columns = df_ana.loc[header_idx]
df_ana = df_ana.loc[header_idx+1:].reset_index(drop=True)
df_ana.columns = df_ana.columns.fillna('') # fill NaN headers ""

# Join column values using whitespace for columns with NaN header
df_ana = df_ana.fillna('')
joined_nan_cols = df_ana.loc[:, ""].apply(lambda x: "".join(x.astype(str)), axis=1)

# capture first nan column pos index
nan_col_idx = [i for i,col in enumerate(df_ana.columns) if col == ""][0]

# remove original nan columns
df_ana = df_ana.drop([''], axis='columns')
df_ana.insert(nan_col_idx, "Comment", joined_nan_cols)

# add comment button 
df_ana.Comment[df_ana.Comment != ''] = df_ana.Comment[df_ana.Comment != ''].apply(comment_button)
df_ana

8,Ratio:,Average,Current,Value?,Comment,Oct 22,Sep 22,Jan 23,Feb 23
0,S&P 500 (P/E),15,19.97,Overvalued,"<button type=""button"" class=""btn btn-secondary...",20.7,21.00,19.97,
1,Shiller P/E,16,28.09,Overvalued,"<button type=""button"" class=""btn btn-secondary...",29.4,31.00,28.09,
2,S&P Price to Book,2.9,3.83,Overvalued,"<button type=""button"" class=""btn btn-secondary...",3.98,4.10,3.83,
3,S&P Price to Sales,1.6,2.31,Overvalued,"<button type=""button"" class=""btn btn-secondary...",2.45,2.60,2.31,
4,S&P Dividend Yield,5.0%,5.0%,Fair,,4.9%,4.8%,5.0%,


# Plotting

In [29]:
import pandas as pd
import numpy as np

# bokeh
from bokeh.io import show, output_notebook
from bokeh.palettes import Category10
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Legend, LabelSet, LegendItem, HoverTool
from bokeh.layouts import column

In [11]:
output_notebook()

Loading BokehJS ...

In [2]:
from dashboard.route.investments import df_plot

In [6]:
df_plot

,Asset_Class,Total_Value
0,Gold/Silver (physical),217695.30
1,Gold/Silver (allocated),1211234.47
2,Business Equity,420335.60
3,Real Estate Equity (IOM/UK),1146588.80
4,Real Estate Equity (US),482000.00
5,Stocks,0.00
6,"Tangibles (Antiques, Art, etc)",35000.00
7,Cash,50000.00
8,NaN,NaN


In [50]:
def pie_chart(
    df: pd.DataFrame, 
    x: str, 
    y: str,
    x_hover: str=None,
    y_hover: str=None, 
    radius: float=0.8,
    x_range: tuple[float, float]=(-1, 1.0),
    percentage_decimal: int=1,
    label_distance: float=3,
    fig_height: int=350,
    background_color: str='#212529',
    pallette: dict=Category10,
    sizing_mode='scale_both',
    hover_tooltip: str='default',
    legend_place: str='right',
    fig_kwargs: dict={},
    wedge_kwargs: dict=dict(line_width=1.5, alpha=0.7),
    legend_kwargs: dict=dict(location='center', click_policy="hide",
                             label_text_color='white', border_line_width=0,
                             inactive_fill_color='#9fcf2e', inactive_fill_alpha=0.15),
    label_kwargs: dict=dict(text_font_size='10pt', text_align='center')
    ):
    
    
    # sort df by "y"
    df = df.sort_values(by=y, ignore_index=True)
    
    # calculate sector start and end angles
    df['angle'] = df[y] / df[y].sum() * 2 * np.pi
    df['cumsum_start'] = df['angle'].cumsum(axis='rows').shift(1).fillna(0)
    df['cumsum_end'] = df['angle'].cumsum(axis='rows')
    
    # calculate y percentages for hover & labels
    df['percentage_number'] = (df[y] / df[y].sum() * 100).round(percentage_decimal)
    df['percentage_hover'] = df['percentage_number'].astype(str)
    df['percentage_label'] = df['percentage_number'].apply(lambda x: "" if x < 5 else f"{x:.{percentage_decimal}f}%")
    
    # project label text coordinates to polar coordinates
    df['label_x_pos'] = np.cos(df['angle'].cumsum() - df['angle'].div(2)) * label_distance * radius/4
    df['label_y_pos'] = np.sin(df['angle'].cumsum() - df['angle'].div(2)) * label_distance * radius/4
    
    # remove assets that are 0
    df = df[df[y] > 0]
    
    # reset dataframe index to start with 0
    df = df.reset_index(drop=True)
    
    print(df)
    
    # init the figure/canvas for the plot
    p = figure(height=fig_height, 
               toolbar_location=None, 
               x_range=x_range,
               y_range=(-1.0, 1.0),
               sizing_mode=sizing_mode,
               **fig_kwargs)
    
    legend_items = []
    for idx, color in enumerate(pallette[df.shape[0]]):
        
        source = ColumnDataSource(df.iloc[idx,:].to_frame().T)
        
        # create the glyphs renderers
        wedge = p.wedge(x=0, y=0, radius=radius, start_angle="cumsum_start", 
                        end_angle="cumsum_end", source=source, **wedge_kwargs,
                        fill_color=color, hover_fill_color=color,
                        line_color=background_color, hover_line_color=background_color,
                        line_alpha=1, hover_alpha=1, hover_line_alpha=1)
        
        label = LabelSet(x='label_x_pos', y='label_y_pos', text='percentage_label',
                         source=source, level='glyph', text_color=background_color, **label_kwargs)
        
        x_hover = x if x_hover is None else x_hover
        y_hover = y if y_hover is None else y_hover
        
        hover_tooltip = hover_tooltip if hover_tooltip != 'default' else \
            f"""
                <div>
                    <p style="margin:0;font-weight:bold;color:grey;">@{x_hover}</p>
                    <p style="padding:0;margin:0;font-weight:bold;">$@{y_hover}{{0,0.00}} (@percentage_hover%)</p>
                </div>
            """
        
        p.add_layout(label)
        p.add_tools(HoverTool(renderers=[wedge],
                              tooltips=hover_tooltip))

        legend_items.append(LegendItem(label=df[x][idx], renderers=[wedge]))

    # legend
    legend = Legend(items=legend_items, **legend_kwargs,
                    background_fill_color=background_color) 
    p.add_layout(legend, place=legend_place)

    
    # figure attributes
    p.toolbar.active_drag = None
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    
    
    p.min_border=0
    p.outline_line_alpha=0
    p.outline_line_width=0
    p.outline_line_color = p.background_fill_color = p.border_fill_color = background_color

    return p

# show(
#     pie_chart(
#         df=df_plot,
#         x='Asset_Class',
#         y='Total_Value',
#         sizing_mode='fixed',
#         legend_place='below',
#         radius=0.5,
#         fig_height=600
        
#     )
# )

# Announcements

In [2]:
from dashboard.route.investments import df_dict, df_hist
import re

df_a = df_dict['announce'].copy()
df_a = df_a.reset_index(drop=True)
df_a = df_a[0].str.split(pat=':', n=1, expand=True)
df_a.columns = ['date', 'text']
no_date_idx = df_a['date'].str.contains(r"property for £980k", case=False).argmax()
df_a.loc[no_date_idx, 'text'] = df_a.loc[no_date_idx, 'date']
df_a.loc[no_date_idx, 'date'] = re.search(r"[A-Z]{1}[a-z]{2}\s202\d{1}", df_a.loc[no_date_idx, 'text'])[0]

df_a


,date,text
0,Feb 2023,Rather than save all your CASH in the bank (u...
1,Jan 2023,Property (Castle) Purchase 23rd Jan 2023. Cash...
2,Jan 2023,As central banks tighten (QT + Interest rate ...
3,Jan 2023,I wouldn't be surprised if we see a small ral...
4,Dec 2022,No change to last month. 2022 financial forec...
5,Nov 2022,Asset prices are now beginning to fall in cor...
6,Oct 2022,Cash is king right now. Assets are finally st...
7,Sep 2022,Cash currently held between multiple bank acc...
8,Aug 2022,"As per last month, I am still accumulating ca..."
9,Jul 2022,I'm following my February plan of going to CA...


# Stocks Watchlist

In [7]:
STOCKS_WATCH_URL = "https://docs.google.com/spreadsheets/d/12-GISr1efphjtpuJLCfQzI2akNXxaJ1iabsG24ib71c/edit#gid=845083323"

# Read in summary DF and drop empty rows
df = read_gsheet(
    STOCKS_WATCH_URL, 
    header=None
)

In [8]:
# find df header row index using regex pattern
header_idx = df.apply(lambda x: x.str.contains("Neil's Value", case=False)).any(axis='columns').argmax()

# separate disclaimer and df
df_disclaimer = df.iloc[:header_idx-1, 0]
df_watch = df.iloc[header_idx:,]

# set first row as header & reset row idxs
df_watch.columns = df_watch.iloc[0].values
df_watch = df_watch.iloc[1:].reset_index(drop=True)

# Generate buttons for 'Notes' column
df_watch.Notes[df_watch.Notes.notna()] = df_watch.Notes[df_watch.Notes.notna()].apply(comment_button)
df_watch = df_watch.fillna("")

# Color Ratings based on category
rating_colormap = {'Sig Undervalued':'green', 'Mod Undervalued':'blue', 'Fair Value':'grey', 'Value Trap?':'red'}
#df_watch["rating_color"] = df_watch.Rating.map(rating_colormap)


In [30]:
df1 = df_disclaimer.copy()

# add period end of the sentence if not so
df1 = df1.to_frame().rename(columns={0: 'info'})
df1.loc[:,'info'] = [x + '.' if x[-1] != '.' else x for x in df1['info']]

df_disc = df1.copy()

df_disc['color'] = ['warning', 'success', 'warning', 'success', 'success', 'danger']
df_disc['color'] = pd.Categorical(df_disc['color'],
                                  categories=['success', 'warning', 'danger'],
                                  ordered=True)
icon_dict = {'warning': 'exclamation-triangle-fill', 'success': 'check-lg', 'danger':'exclamation-octagon-fill'}
df_disc['icon_id'] = df_disc['color'].map(icon_dict)
df_disc = df_disc.sort_values('color')



,info,color,icon_id
1,Please make your own copy of the stocks sectio...,success,check-lg
3,These stocks were found using the techniques t...,success,check-lg
4,"Neil's Stocks Strategy: 80% into Index funds, ...",success,check-lg
0,These Valuations are for the purpose of Long T...,warning,exclamation-triangle-fill
2,NOTE! Many of the metrics below have to be man...,warning,exclamation-triangle-fill
5,"AT THIS TIME, I AM OUT OF ALL STOCKS. PLEASE D...",danger,exclamation-octagon-fill
